## ETL com Python
#### Utilizando ChatGPT e API DIO Santanderdevweek2023

Repositório da API: https://github.com/digitalinnovationone/santander-dev-week-2023-api

Para esse exercício, foi solicitado a criação de um novo cadastro. Foi acessado a Swagger da API e efetuado novo cadastro cuja id resultante foi a 1300.

https://sdw-2023-prd.up.railway.app/swagger-ui/index.html

Para prosseguir com as ações, optei aproveitar a base de dados já existente, efetuando o download dos cadastros em formato json.

Para a utilização da API do ChatGPT, antes, se faz necessário a instalação da biblioteca da openai:

In [15]:
# !pip install openai

### Imports

In [1]:
import pandas as pd
import requests
import json

import openai

In [2]:
# endereço da API da DIO (elaborada na Dev Week 2023)
sdw2023_api_url = 'https://sdw-2023-prd.up.railway.app'

### Funções

In [3]:
def get_user(id):    
    '''Recupera as informações do usuário a partir do valor de id fornecido.
    Retorna o json caso resposta seja status code 200 de id existente, 
    caso contrário None
    arg: id: integer aceita somente valores das id cadastradas'''
    
    response = requests.get(f'{sdw2023_api_url}/users/{id}')
    return response.json() if response.status_code == 200 else None


def generate_ai_news(user):
    '''Envia requisição à API da OpenAI para a obtenção de mensagem personalizada
    Atribuindo o nome do usuário.
    arg: user: json'''
    
    completion = openai.ChatCompletion.create(        
        model="gpt-4",
        messages=[
            {
                "role": "system", "content": "Você é um especialista em marketing bancário"
            },
            {
                "role": "user", 
                "content": f"Crie uma mensagem para {user['name']} sobre a \
                importância dos investimentos (máximo de 100 caracteres)"
            }
        ]
    )
    
    responseChatGPT = completion.choices[0].message.content.strip('\"')
    
    
def generate_ai_news_teste(user):
    '''Função auxiliar para caso não houver cadastro ativo na OpenIA ou outros semelhantes.
    Retorna string
    arg: user: json'''
    
    return f"{user['name']}, vai ganhar na mega-sena mas, primeiro, você deve apostar."


def update_user(user):
    '''Função efetua requisição PUT para a edição de valores na API.
    Retorna boolean: True/False. Verdadeiro se resposta à requisição for status code 200.
    arg: user: json'''
    
    response = requests.put(f"{sdw2023_api_url}/users/{user['id']}", json=user)
    return True if response.status_code == 200 else False

### Extract
Extrair a lista de IDs de usuários a partir do arquivo*. Após, é efetuada uma requisição GET para obtenção dos dados de cada usuário.

obs: optei por aproveitar as IDs cadastradas anteriormente (além de criar um novo usuário). Portanto, em determinado momento, obtive a base de dados efetuando uma requisição GET /users e salvei no arquivo SDW2023.json.

In [4]:
data = pd.read_json('SDW2023.json')

#### Conhecendo os dados

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1110 entries, 0 to 1109
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        1110 non-null   int64 
 1   name      1110 non-null   object
 2   account   1110 non-null   object
 3   card      1110 non-null   object
 4   features  1110 non-null   object
 5   news      1110 non-null   object
dtypes: int64(1), object(5)
memory usage: 52.2+ KB


Há um total de 1110 cadastros e não há valores nulos. Entretanto, em etapas seguintes, utilizarei uma pequena amostra para fins práticos.

In [6]:
data = data.sort_values('id')

In [7]:
data[['id', 'name']].head(3)

,id,name
7,1,Devweekerson
11,13,anguile
12,14,Maria


In [8]:
data[['id', 'name']].tail(3)

,id,name
1107,1298,Ricardo
1108,1299,Santos
1109,1300,Sir Pyterson II


In [9]:
user_ids = data['id'].tolist()
# print(user_ids)

In [10]:
data.loc[data['id'] == 1]

,id,name,account,card,features,news
7,1,Devweekerson,"{'id': 1, 'number': '01.097954-4', 'agency': '...","{'id': 1, 'number': 'xxxx xxxx xxxx 1111', 'li...","[{'id': 2, 'icon': 'https://digitalinnovationo...","[{'id': 1, 'icon': 'https://digitalinnovationo..."


Obs:<br>
Como não desejo recuperar os dados de todos os usuários cadastrados (o que levaria tempo), utilizarei apenas os cinco últimos cadastros. Atualmente (02/08/2023) refere-se a um total de cinco cadastros incluindo o criado por mim (id=1300).
Também os escolhi pois, inicialmente, estes não estavam preenchidos o campo "news" (posteriormente ocorreram acréscimos/alterações, possivelmente de seu criador original).

In [11]:
list_user = [user for id in user_ids[-5:] if (user := get_user(id)) is not None]

In [12]:
print(json.dumps(list_user, indent=2))

[
  {
    "id": 1288,
    "name": "Sherlock",
    "account": {
      "id": 9,
      "number": "00345-1",
      "agency": "0001",
      "balance": 0.0,
      "limit": 500.0
    },
    "card": {
      "id": 6,
      "number": "0000 1234 9854 1234",
      "limit": 1000.0
    },
    "features": [
      {
        "id": 437,
        "icon": "string",
        "description": "string"
      }
    ],
    "news": [
      {
        "id": 2674,
        "icon": "https://digitalinnovationone.github.io/santander-dev-week-2023-api/icons/credit.svg",
        "description": "Sherlock, vai ganhar na mega-sena mas, primeiro, voc\u00ea deve apostar."
      }
    ]
  },
  {
    "id": 1297,
    "name": "Paulo",
    "account": {
      "id": 1331,
      "number": "5565",
      "agency": "Santander",
      "balance": 0.0,
      "limit": 1000.0
    },
    "card": {
      "id": 1234,
      "number": "xxxx xxxxx 7777",
      "limit": 500.0
    },
    "features": [
      {
        "id": 438,
        "icon": "string"

### Transform 
Gerar uma mensagem de marketing personalizada para cada usuário.

Documentação oficial da API OpenAI: https://platform.openai.com/docs/api-reference/introduction
Informações sobre o período gratuito: https://help.openai.com/en/articles/4936830

Para gerar uma API Key:
1. Crie uma conta na OpenAI
2. Acesse a seção "API Keys"
3. Clique em "Create API Key"

Link direto: https://platform.openai.com/account/api-keys
Substitua o texto TODO por sua API Key da OpenAI, ela será salva como uma variável de ambiente.

In [13]:
openai.api_key = '' # preencher com a própria chave

Para não gerar respostas JSON grandes em demasia, ao invés do append para a cada vez incluir nova mensagem, optei para reaproveitar a id da mensagem já existente (geralmente a primeira).

In [14]:
for user in list_user:
    news = generate_ai_news_teste(user)
    print(news)

    user['news'][0] = {
            "id": user['news'][0]['id'],
            "icon": "https://digitalinnovationone.github.io/santander-dev-week-2023-api/icons/credit.svg",
            "description": news
        }    

Sherlock, vai ganhar na mega-sena mas, primeiro, você deve apostar.
Paulo, vai ganhar na mega-sena mas, primeiro, você deve apostar.
Ricardo, vai ganhar na mega-sena mas, primeiro, você deve apostar.
Santos, vai ganhar na mega-sena mas, primeiro, você deve apostar.
Sir Pyterson II, vai ganhar na mega-sena mas, primeiro, você deve apostar.


### Load
Atualiza o valor de "news" de cada usuário na API com a nova mensagem gerada.

In [15]:
for user in list_user:
    success = update_user(user)
    print(f"User {user['name']} updated? {success}!")

User Sherlock updated? True!
User Paulo updated? True!
User Ricardo updated? True!
User Santos updated? True!
User Sir Pyterson II updated? True!


### Considerações

Este pequeno projeto há muitas possibilidades de melhorias, sejam em desempenho, abraangência de escopo ou simplesmente estéticos. Todavia, me atentarei ao essencial.

Uma sugestão de melhoria, não seria para este projeto em sim mas a extensão da API com a inclusão do campo "estado" para cada usuário cadastrado. 
O que aumentaria o valor da API utilizada para fins didátivos. Permitindo aos alunos/interessados, desenvolverem análises abraangentes e justificaria o emprego de visualização de dados. 